# Supporting Microsoft’s GraphRAG: Part 3

In the [previous section](../msft_graphrag_2), we have created a graph in TigerGraph, and loaded the CSV data into it.

Now, let’s use Jupyter Notebook to explore the graph data and perform graph analysis.

To run this Jupyter Notebook, you can download the original `.ipynb` file from [msft_graphrag_3.ipynb](https://github.com/xuanleilin/tigergraphx/tree/main/docs/graphrag/msft_graphrag_3.ipynb).

---

## Get the Graph from TigerGraph
Since the graph has already been created in TigerGraph, redefining its schema is unnecessary. Instead, you can provide the graph name to retrieve it. TigerGraphX will verify if the graph exists in TigerGraph and, if it does, will return the corresponding graph.

In [1]:
from tigergraphx import Graph
connection = {
    "host": "http://127.0.0.1",
    "username": "tigergraph",
    "password": "tigergraph",
}
G = Graph.from_db("GraphRAG", connection)

2025-01-05 23:30:15,203 - tigergraphx.core.graph.base_graph - INFO - Creating schema for graph GraphRAG...
2025-01-05 23:30:15,223 - tigergraphx.core.graph.base_graph - INFO - Schema created successfully.


## Display the Graph Schema

Let's retrieve the graph schema using the `get_schema` method. The output is a Python dictionary containing three keys: `"graph_name"`, `"nodes"`, and `"edges"`. We'll print each of them individually to explore the schema details.
### Retrieve the Graph Schema and Display the Graph Name

In [2]:
schema = G.get_schema()
print(schema["graph_name"])

GraphRAG


### Display the Node Tyeps

In [3]:
for node in schema["nodes"].items():
    print(node)

('Document', {'primary_key': 'id', 'attributes': {'title': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}, 'id': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}}, 'vector_attributes': {}})
('TextUnit', {'primary_key': 'id', 'attributes': {'text': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}, 'n_tokens': {'data_type': <DataType.UINT: 'UINT'>, 'default_value': None}, 'id': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}}, 'vector_attributes': {}})
('Entity', {'primary_key': 'id', 'attributes': {'human_readable_id': {'data_type': <DataType.UINT: 'UINT'>, 'default_value': None}, 'name': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}, 'entity_type': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}, 'description': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}, 'id': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}}, 'vector_attributes': {}})
('R

### Display the Edge Types

In [4]:
for edge in schema["edges"].items():
    print(edge)

('document_contains_text_unit', {'is_directed_edge': False, 'from_node_type': 'Document', 'to_node_type': 'TextUnit', 'edge_identifier': set(), 'attributes': {}})
('text_unit_contains_entity', {'is_directed_edge': False, 'from_node_type': 'TextUnit', 'to_node_type': 'Entity', 'edge_identifier': set(), 'attributes': {}})
('text_unit_contains_relationship', {'is_directed_edge': False, 'from_node_type': 'TextUnit', 'to_node_type': 'Relationship', 'edge_identifier': set(), 'attributes': {}})
('relationship_source', {'is_directed_edge': False, 'from_node_type': 'Relationship', 'to_node_type': 'Entity', 'edge_identifier': set(), 'attributes': {}})
('relationship_target', {'is_directed_edge': False, 'from_node_type': 'Relationship', 'to_node_type': 'Entity', 'edge_identifier': set(), 'attributes': {}})
('community_contains_entity', {'is_directed_edge': False, 'from_node_type': 'Community', 'to_node_type': 'Entity', 'edge_identifier': set(), 'attributes': {}})
('community_contains_relationship

## Display Node and Edge Counts

Gain deeper insights into the graph by exploring details such as the total number of nodes and the count of nodes for each node type.

### Display the Total Number of Nodes

In [5]:
G.number_of_nodes()

371

### Display the Count of Nodes for Each Node Type

In [6]:
for node_type in schema["nodes"]:
    print(f"{node_type}: {G.number_of_nodes(node_type)}")

Document: 1
TextUnit: 42
Entity: 138
Relationship: 168
Community: 22


### Display the Total Number of Edges

In [7]:
G.number_of_edges()

1545

### Display the Count of Edges for Each Edge Type

In [8]:
for edge_type in schema["edges"]:
    print(f"{edge_type}: {G.number_of_edges(edge_type)}")

document_contains_text_unit: 42
text_unit_contains_entity: 274
text_unit_contains_relationship: 238
relationship_source: 168
relationship_target: 168
community_contains_entity: 236
community_contains_relationship: 419


## Retrieve Sample Nodes from the Graph

In [9]:
G.get_nodes(node_type="Entity", limit=2)

,v_id,v_type,human_readable_id,entity_type,name,description,id
0,473502492d0a419981fed4fbc1493832,Entity,69,PERSON,THE THREE MISS FEZZIWIGS,"Daughters of Mr. and Mrs. Fezziwig, described ...",473502492d0a419981fed4fbc1493832
1,6fb90dc954fe40d5969f7532a66376e9,Entity,108,PERSON,WANT,"Want is depicted as a girl, symbolizing povert...",6fb90dc954fe40d5969f7532a66376e9


In [10]:
G.get_nodes(node_type="Relationship", limit=2)

,v_id,v_type,human_readable_id,rank,weight,description,id
0,9953ed9db4c5418e8bf9fee18032c0da,Relationship,63,9,20,Fezziwig and Mrs. Fezziwig share a close perso...,9953ed9db4c5418e8bf9fee18032c0da
1,e9953a1648364e878d835bc6bcc0d3ef,Relationship,37,5,1,The activities and cheer that the Ghost of Chr...,e9953a1648364e878d835bc6bcc0d3ef


In [11]:
G.get_nodes(node_type="Community", limit=2)

,v_id,v_type,summary,level,full_content,rank,id,rank_explanation,title
0,12,Community,This report delves into the interconnected rel...,1,# The Transformation of Ebenezer Scrooge: A Ch...,8.5,12,The high impact severity rating reflects the p...,Community 12
1,7,Community,This report explores the network surrounding P...,0,# Project Gutenberg and the Digital Disseminat...,8.5,7,The high impact severity rating reflects Proje...,Community 7


---

## What’s Next?

- [Supporting Microsoft’s GraphRAG: Part 4](../msft_graphrag_4): Perform queries using GSQL and Python-native TigerGraphX, with global and local context builders.

---

Start transforming your GraphRAG workflows with the power of **TigerGraphX** today!